In [1]:
import os
import numpy as np
import pandas as pd
import ROOT

import ctypes
from pprint import pprint

Welcome to JupyROOT 6.26/08


In [82]:
ERA = "2016"
CHANNEL = "ZGamma3Mu"
MEASURE = "LowPT"
histkey = "ZCand/mass"

WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"

DataStream = "DoubleMuon"
Conversion  = ["DYJets", "ZGToLLG"]
VV          = ["WZTo3LNu_amcatnlo","ZZTo4L_powheg"]
ttX         = ["ttWToLNu", "ttZToLLNuNu", "ttHToNonbb", "tZq", "tHq"]
Rare        = ["WWW", "WWZ", "WZZ", "ZZZ", "TTTT", "VBF_HToZZTo4L", "GluGluHToZZTo4L"] # no WWG / TTG -> conversion
MCSamples   = Conversion + VV + ttX + Rare

Systematics = [["NonpromptUp", "NonpromptDown"],
               ["L1PrefireUp", "L1PrefireDown"],
               ["PileupReweightUp", "PileupReweightDown"],
               ["MuonIDSFUp", "MuonIDSFDown"],
               ["DblMuTrigSFUp", "DblMuTrigSFDown"],
               ["JetResUp", "JetResDown"],
               ["JetEnUp", "JetEnDown"],
               ["ElectronResUp", "ElectronResDown"],
               ["ElectronEnUp", "ElectronEnDown"],
               ["MuonEnUp", "MuonEnDown"],
               ["HeavyTagUpCorr", "HeavyTagDownCorr"],
               ["HeavyTagUpUnCorr", "HeavyTagDownUnCorr"],
               ["LightTagUpCorr", "LightTagDownCorr"],
               ["LightTagUpUnCorr", "LightTagDownUnCorr"]
              ]

In [83]:
sample = {}

# make index
indexCol = ["Sample", "Central", "Stat"]
for syst in Systematics:
    indexCol.append(syst[0])
    indexCol.append(syst[1])
indexCol.append("Total")

for index in indexCol:
    sample[index] = []

In [84]:
def estimateTotalErr(sample, sampleDict):
    # find the index of the sample
    idx = sampleDict["Sample"].index(sample)
    central = sampleDict['Central'][idx]
    totalErr = np.power(sampleDict['Stat'][idx], 2)
    for syst in Systematics:
        if not sampleDict[syst[0]][idx]: continue
        systUp = abs(sampleDict[syst[0]][idx] - central)
        systDown = abs(sampleDict[syst[1]][idx] - central)
        totalErr += np.power(max(systUp, systDown), 2)
    
    return np.sqrt(totalErr)

In [85]:
# data
f = ROOT.TFile.Open(f"{WORKDIR}/data/MeasConversion/{ERA}/Skim3Mu__/DATA/MeasConversion_SkimTree_SS2lOR3l_DoubleMuon.root")

sample['Sample'].append("data")
hData = f.Get(f"{CHANNEL}/{MEASURE}/Central/{histkey}")
hData.SetDirectory(0)

stat = ctypes.c_double()
data = hData.IntegralAndError(0, hData.GetNbinsX()+1, stat)
sample['Central'].append(data)
sample['Stat'].append(stat.value)

for index in indexCol[3:-1]:
    sample[index].append(False)
sample['Total'].append(estimateTotalErr("data", sample))

f.Close()

In [86]:
# Nonprompt
f = ROOT.TFile.Open(f"{WORKDIR}/data/MeasConvMatrix/{ERA}/Skim3Mu__/DATA/MeasConvMatrix_SkimTree_SS2lOR3l_DoubleMuon.root")

sample['Sample'].append("nonprompt")
hFake = f.Get(f"{CHANNEL}/{MEASURE}/Central/{histkey}"); hFake.SetDirectory(0)
hFakeUp = f.Get(f"{CHANNEL}/{MEASURE}/NonpromptUp/{histkey}"); hFakeUp.SetDirectory(0)
hFakeDown = f.Get(f"{CHANNEL}/{MEASURE}/NonpromptDown/{histkey}"); hFakeDown.SetDirectory(0)

stat = ctypes.c_double()
fake = hFake.IntegralAndError(0, hFake.GetNbinsX(), stat)
sample['Central'].append(fake)
sample['Stat'].append(stat.value)

# systematics
sample['NonpromptUp'].append(hFakeUp.Integral())
sample['NonpromptDown'].append(hFakeDown.Integral())

for index in indexCol[5:-1]:
    sample[index].append(False)
sample['Total'].append(estimateTotalErr("nonprompt", sample))

f.Close()

In [87]:
for mc in MCSamples:
    fkey = f"{WORKDIR}/data/MeasConversion/{ERA}/Skim3Mu__ScaleVar__WeightVar__/MeasConversion_SkimTree_SS2lOR3l_{mc}.root"
    if not os.path.exists(fkey):
        fkey = f"{WORKDIR}/data/MeasConversion/{ERA}/Skim3Mu__ScaleVar__WeightVar__/MeasConversion_{mc}.root"
    
    f = ROOT.TFile.Open(fkey)
    try:
        hCentral = f.Get(f"{CHANNEL}/{MEASURE}/Central/{histkey}")
        hCentral.SetDirectory(0)
    except Exception as e:
        print(e)
        print(mc)
        continue
    
    sample['Sample'].append(mc) 
    # fill each row
    stat = ctypes.c_double()
    central = hCentral.IntegralAndError(0, hCentral.GetNbinsX()+1, stat)
    sample['Central'].append(central)
    sample['Stat'].append(stat.value)
    sample['NonpromptUp'].append(False)
    sample['NonpromptDown'].append(False)
    
    print(mc)
    for index in indexCol[5:-1]:
        try:
            hSyst = f.Get(f"{CHANNEL}/{MEASURE}/{index}/{histkey}")
            hSyst.SetDirectory(0)
            sample[index].append(hSyst.Integral())
        except Exception as e:
            sample[index].append(0.)
    f.Close()
    
    sample['Total'].append(estimateTotalErr(mc, sample))

DYJets
ZGToLLG
WZTo3LNu_amcatnlo
ZZTo4L_powheg
ttWToLNu
ttZToLLNuNu
ttHToNonbb
tZq
tHq
WWW
WWZ
WZZ
ZZZ
TTTT
VBF_HToZZTo4L
GluGluHToZZTo4L


In [88]:
# make dataframe
df = pd.DataFrame(sample)
df.set_index("Sample", inplace=True)
df = df.transpose()

# measure scale factor
ConvSFDict = {}

# central value
nData = df.loc['Central', 'data']
nConv = df.loc['Central', "DYJets"] if MEASURE == "LowPT" else df.loc['Central', "ZGToLLG"]
nPred = df.loc['Central', 'nonprompt']
for mc in MCSamples:
    if mc in Conversion: continue
    if not mc in df.columns: continue
    nPred += df.loc['Central', mc]
ConvSFDict['Central'] = (nData - nPred) / nConv

In [89]:
pprint(df)

Sample                data  nonprompt     DYJets    ZGToLLG WZTo3LNu_amcatnlo  \
Central               70.0  27.250235  26.458121  13.279151          6.246837   
Stat                8.3666   3.882995  10.201383    1.13939          0.408635   
NonpromptUp          False  46.391504      False      False             False   
NonpromptDown        False  21.673581      False      False             False   
L1PrefireUp          False      False   26.28211  13.167656           6.19601   
L1PrefireDown        False      False    26.6162  13.381335          6.295419   
PileupReweightUp     False      False  26.043739  13.811858           6.35725   
PileupReweightDown   False      False  27.626247  12.981588          6.171954   
MuonIDSFUp           False      False  27.517868  13.802795          6.486721   
MuonIDSFDown         False      False  25.421725  12.766163          6.011871   
DblMuTrigSFUp        False      False  26.467112   13.28275          6.248867   
DblMuTrigSFDown      False  

In [90]:
thisConv = "DYJets" if MEASURE == "LowPT" else "ZGToLLG"
# stat
dNconv = df.loc["Stat", thisConv]
dNpred = df.loc["Stat", 'nonprompt']
for mc in MCSamples:
    if mc in Conversion: continue
    if not mc in df.columns: continue
    dNpred += df.loc["Stat", mc]
ConvSFDict["Stat"] = -(dNpred / nConv) + (nPred / np.power(nConv, 2))*dNconv

# systematics
for syst in indexCol[3:-1]:
    dNconv = 0.
    if df.loc[syst, thisConv]: dNconv +=  df.loc[syst, thisConv] - df.loc['Central', thisConv]
    
    dNpred = 0.
    if df.loc[syst, 'nonprompt']: dNpred += df.loc[syst, 'nonprompt'] - df.loc['Central', 'nonprompt']
    for mc in MCSamples:
        if mc in Conversion: continue
        if not mc in df.columns: continue
        if df.loc[syst, mc]: dNpred += df.loc[syst, mc] - df.loc['Central', mc]
    ConvSFDict[syst] = -(dNpred / nConv) + (nPred / np.power(nConv, 2))*dNconv
    
totalErr = np.power(ConvSFDict['Stat'], 2)
for syst in Systematics:
    thisErr = max( abs(ConvSFDict[syst[0]]), abs(ConvSFDict[syst[1]]) )
    totalErr += np.power(thisErr, 2)
ConvSFDict["Total"] = np.sqrt(totalErr)

In [91]:
pprint(ConvSFDict)

{'Central': 0.9018330563709817,
 'DblMuTrigSFDown': -0.0003625264040598783,
 'DblMuTrigSFUp': 0.0003583534921076857,
 'ElectronEnDown': 0.0,
 'ElectronEnUp': 0.0,
 'ElectronResDown': 0.0,
 'ElectronResUp': 0.0,
 'HeavyTagDownCorr': 0.001599595241567664,
 'HeavyTagDownUnCorr': 0.0016434845988705714,
 'HeavyTagUpCorr': -0.0016468607335040917,
 'HeavyTagUpUnCorr': -0.001702626259982297,
 'JetEnDown': -0.06362399448855421,
 'JetEnUp': -0.05203658664585869,
 'JetResDown': -0.0027831628292310125,
 'JetResUp': 0.03151084164975449,
 'L1PrefireDown': 0.005029922138160196,
 'L1PrefireUp': -0.005765517577669091,
 'LightTagDownCorr': 0.013244555564530011,
 'LightTagDownUnCorr': 0.013048867601051164,
 'LightTagUpCorr': -0.013156912915614296,
 'LightTagUpUnCorr': -0.01297400592834668,
 'MuonEnDown': 0.000994950152387817,
 'MuonEnUp': 0.0006242548564609832,
 'MuonIDSFDown': -0.04080734213612311,
 'MuonIDSFUp': 0.04175323547254721,
 'NonpromptDown': 0.21077286804827414,
 'NonpromptUp': -0.723455335215